In [9]:
import pandas as pd
import os
import glob
import json
import csv
import cassandra
import re
import numpy as np

In [10]:
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

# keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)
    
# connect to keyspace
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

In [11]:
query = "select artist, song_title, song_duration from music_app_session_history WHERE session_id=338 and item_in_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.song_duration)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [12]:
query = "SELECT artist, song, user_first_name, user_last_name from music_app_user_history WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.user_first_name, row.user_last_name)

Keep On Keepin' On Down To The Bone Sylvie Cruz
Greece 2000 Three Drives Sylvie Cruz
Kilometer Sebastien Tellier Sylvie Cruz
Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Lonnie Gordon Sylvie Cruz


In [13]:
query = "SELECT user_first_name, user_last_name from music_app_song_history WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_first_name + " " + row.user_last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


In [14]:
session.shutdown()
cluster.shutdown()